In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import glob
import os
from ntpath import basename, splitext
 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
 
from scipy.signal import savgol_filter
import warnings
warnings.filterwarnings("ignore")

In [88]:
files = glob.glob(os.path.join(path,'*.csv'))

In [89]:
# Please use meaningful variable names:
basenames = []

for doc_path in files:

    # Please don't reinvent the wheel.
    # Use the builtin path handling functions.
    # File naming has a lot of exceptions and weird cases 
    # (particularly on Windows).
    file_name = basename(doc_path)
    file_basename, extension = splitext(file_name)
    if extension == ".csv":
        basenames.append(file_basename)

print(basenames)

['BIOHEDGE-SOG096976_2022-06-14T04-55-33', 'BIOHEDGE-SOG096977_2022-06-14T04-56-28', 'BIOHEDGE-SOG096978_2022-06-14T04-57-14', 'BIOHEDGE-SOG096979_2022-06-14T04-58-18', 'BIOHEDGE-SOG096980_2022-06-14T04-59-03', 'BIOHEDGE-SOG097001_2022-06-14T04-09-13', 'BIOHEDGE-SOG097002_2022-06-14T04-10-06', 'BIOHEDGE-SOG097004_2022-06-14T04-13-48', 'BIOHEDGE-SOG097005_2022-06-14T04-15-50', 'BIOHEDGE-SOG097006_2022-06-14T04-18-04', 'BIOHEDGE-SOG097007_2022-06-14T04-19-38', 'BIOHEDGE-SOG097008_2022-06-14T04-20-27', 'BIOHEDGE-SOG097009_2022-06-14T04-21-15', 'BIOHEDGE-SOG097010_2022-06-14T04-22-22', 'BIOHEDGE-SOG097011_2022-06-14T04-23-13', 'BIOHEDGE-SOG097012_2022-06-14T04-24-21', 'BIOHEDGE-SOG097013_2022-06-14T04-27-53', 'BIOHEDGE-SOG097014_2022-06-14T04-29-13', 'BIOHEDGE-SOG097015_2022-06-14T04-30-05', 'BIOHEDGE-SOG097016_2022-06-14T04-30-52', 'BIOHEDGE-SOG097017_2022-06-14T04-31-39', 'BIOHEDGE-SOG097018_2022-06-14T04-32-28', 'BIOHEDGE-SOG097527_2022-06-14T04-48-37', 'BIOHEDGE-SOG097528_2022-06-14T04

In [158]:
basenames_new  = [i.split('_2022', 1)[0] for i in basenames]
id_names = [i.split('BIOHEDGE-', 1)[1] for i in basenames_new]
id_names.insert(0, 'Wavenumber')
id_names

['Wavenumber',
 'SOG096976',
 'SOG096977',
 'SOG096978',
 'SOG096979',
 'SOG096980',
 'SOG097001',
 'SOG097002',
 'SOG097004',
 'SOG097005',
 'SOG097006',
 'SOG097007',
 'SOG097008',
 'SOG097009',
 'SOG097010',
 'SOG097011',
 'SOG097012',
 'SOG097013',
 'SOG097014',
 'SOG097015',
 'SOG097016',
 'SOG097017',
 'SOG097018',
 'SOG097527',
 'SOG097528',
 'SOG097537',
 'SOG097538',
 'SOG097547',
 'SOG097548']

In [175]:
dataframes = [pd.read_csv(f) for f in files]

In [92]:
def reduce(function, iterable, initializer=None):
    it = iter(iterable)
    if initializer is None:
        value = next(it)
    else:
        value = initializer
    for element in it:
        value = function(value, element)
    return value

df_merged = reduce(lambda left, right: pd.merge(left, right, on = 'Wavenumber', how = 'outer'), dataframes)

In [93]:
df_merged.columns = id_names

In [95]:
df_merged.shape

(3596, 29)

In [168]:
df_merged.head()

,Wavenumber,SOG096976,SOG096977,SOG096978,SOG096979,SOG096980,SOG097001,SOG097002,SOG097004,SOG097005,...,SOG097015,SOG097016,SOG097017,SOG097018,SOG097527,SOG097528,SOG097537,SOG097538,SOG097547,SOG097548
0,650,5.339301,1.499050,3.488021,4.967658,4.955530,1.841816,3.859263,3.003718,0.000010,...,1.330091,2.368278,0.332137,2.143862,4.373419,8.976367,3.873219,6.039929,2.319238,2.228072
1,651,5.813316,3.559210,5.705764,6.437532,6.727562,1.928635,4.294497,3.830015,1.508527,...,1.697372,2.618339,0.573760,2.764627,5.391403,8.285902,3.793475,6.450260,3.457124,1.941637
2,652,5.191623,4.129518,4.157185,4.983297,6.320817,0.000010,2.972787,3.214691,3.056192,...,2.037935,0.937674,0.819564,2.809715,3.304479,5.114044,2.008535,6.175025,3.275428,0.521531
3,653,3.831078,4.083352,1.016578,2.275224,4.129225,0.000010,0.099812,1.872065,2.263256,...,2.349417,0.000010,0.842188,2.599404,0.614418,1.716332,0.389996,6.579527,3.157746,0.000010
4,654,3.923971,5.378190,0.280045,1.839424,3.191216,0.000010,0.000010,1.822128,1.499067,...,4.259833,0.682313,2.033175,3.481404,0.901971,1.372303,1.617383,8.422524,4.890372,2.251391


In [167]:
df_merged['Wavenumber'] = df_merged['Wavenumber'].round(0).astype('int')

In [159]:
df_merged.to_csv("df_merged.csv")

In [147]:
df1_trans = df_merged.set_index('Wavenumber').T
df1_trans.insert(0, 'Sample ID', df1_trans.index)

In [148]:
df1_trans.reset_index(inplace = True, drop = True)

In [153]:
df1_trans.head()

Wavenumber,Sample ID,650.4205,651.3523,652.2841,653.216,654.1478,655.0796,656.0115,656.9433,657.8751,...,3991.9788,3992.9107,3993.8425,3994.7743,3995.7062,3996.638,3997.5698,3998.5017,3999.4335,4000.3653
0,SOG096976,5.339301,5.813316,5.191623,3.831078,3.923971,4.778427,4.636455,4.484315,5.588830,...,80.128433,79.881936,80.480237,81.152621,81.333001,81.117958,80.776530,80.419133,80.178999,80.218194
1,SOG096977,1.499050,3.559210,4.129518,4.083352,5.378190,6.601518,5.823354,4.488095,4.746883,...,76.479494,76.408214,76.521606,76.255129,75.782400,75.698871,76.067100,76.399546,76.458680,76.427402
2,SOG096978,3.488021,5.705764,4.157185,1.016578,0.280045,0.839366,0.000010,0.000010,0.000010,...,59.837764,59.247717,58.874832,58.213424,57.338738,57.030356,57.635755,58.516150,59.014094,59.246489
3,SOG096979,4.967658,6.437532,4.983297,2.275224,1.839424,2.939722,3.074332,2.996978,4.206669,...,89.548594,89.643503,90.027703,90.143119,89.823094,89.282078,88.631148,87.952851,87.559034,87.636642
4,SOG096980,4.955530,6.727562,6.320817,4.129225,3.191216,3.649213,3.278726,2.391073,3.021998,...,72.295031,72.076491,71.766228,70.971789,69.960453,69.433274,69.552605,69.823534,69.954061,70.139841


In [173]:
df1_trans['Sample ID'].tolist()

['SOG096976',
 'SOG096977',
 'SOG096978',
 'SOG096979',
 'SOG096980',
 'SOG097001',
 'SOG097002',
 'SOG097004',
 'SOG097005',
 'SOG097006',
 'SOG097007',
 'SOG097008',
 'SOG097009',
 'SOG097010',
 'SOG097011',
 'SOG097012',
 'SOG097013',
 'SOG097014',
 'SOG097015',
 'SOG097016',
 'SOG097017',
 'SOG097018',
 'SOG097527',
 'SOG097528',
 'SOG097537',
 'SOG097538',
 'SOG097547',
 'SOG097548']

In [172]:
toc = pd.read_csv("BioHedgeTOC.csv")
print(toc['Sample ID'].tolist())

['SOG096976',
 'SOG096977',
 'SOG096978',
 'SOG096979',
 'SOG096980',
 'SOG097001',
 'SOG097002',
 'SOG097004',
 'SOG097005',
 'SOG097006',
 'SOG097007',
 'SOG097008',
 'SOG097009',
 'SOG097010',
 'SOG097011',
 'SOG097012',
 'SOG097013',
 'SOG097014',
 'SOG097015',
 'SOG097016',
 'SOG097517',
 'SOG097518',
 'SOG097527',
 'SOG097528',
 'SOG097537',
 'SOG097538',
 'SOG097547',
 'SOG097548']

In [155]:
df_def = pd.merge(toc, df1_trans, on = 'Sample ID')
df_def

,Sample ID,%TOC,650.4205,651.3523,652.2841,653.216,654.1478,655.0796,656.0115,656.9433,...,3991.9788,3992.9107,3993.8425,3994.7743,3995.7062,3996.638,3997.5698,3998.5017,3999.4335,4000.3653
0,SOG096976,3.593,5.339301,5.813316,5.191623,3.831078,3.923971,4.778427,4.636455,4.484315,...,80.128433,79.881936,80.480237,81.152621,81.333001,81.117958,80.776530,80.419133,80.178999,80.218194
1,SOG096977,2.341,1.499050,3.559210,4.129518,4.083352,5.378190,6.601518,5.823354,4.488095,...,76.479494,76.408214,76.521606,76.255129,75.782400,75.698871,76.067100,76.399546,76.458680,76.427402
2,SOG096978,2.117,3.488021,5.705764,4.157185,1.016578,0.280045,0.839366,0.000010,0.000010,...,59.837764,59.247717,58.874832,58.213424,57.338738,57.030356,57.635755,58.516150,59.014094,59.246489
3,SOG096979,1.114,4.967658,6.437532,4.983297,2.275224,1.839424,2.939722,3.074332,2.996978,...,89.548594,89.643503,90.027703,90.143119,89.823094,89.282078,88.631148,87.952851,87.559034,87.636642
4,SOG096980,1.303,4.955530,6.727562,6.320817,4.129225,3.191216,3.649213,3.278726,2.391073,...,72.295031,72.076491,71.766228,70.971789,69.960453,69.433274,69.552605,69.823534,69.954061,70.139841
5,SOG097001,3.529,1.841816,1.928635,0.000010,0.000010,0.000010,0.805033,2.492221,3.072716,...,33.427626,33.515342,33.586071,33.443948,32.946649,32.495029,32.559067,32.882884,32.859553,32.516591
6,SOG097002,2.485,3.859263,4.294497,2.972787,0.099812,0.000010,0.000010,0.000010,0.054350,...,48.526384,48.506722,48.414948,48.222553,47.852507,47.549975,47.563092,47.645377,47.457735,47.242293
7,SOG097004,1.492,3.003718,3.830015,3.214691,1.872065,1.822128,2.046759,0.788455,0.000010,...,62.694093,62.620840,62.660043,62.673517,62.446692,62.144689,62.099655,62.291412,62.447114,62.507168
8,SOG097005,0.933,0.000010,1.508527,3.056192,2.263256,1.499067,2.168342,2.354969,1.347949,...,22.394004,22.769797,22.737383,22.175932,21.212699,20.451714,20.373873,20.651162,20.666591,20.484950
9,SOG097006,0.720,0.696772,2.471819,1.559476,0.000010,0.276298,3.276061,5.090133,4.996736,...,40.750809,40.925682,41.275677,41.603750,41.502022,41.134729,41.046043,41.295368,41.469440,41.438030


In [160]:
df_def.to_csv('df_def.csv')